In [ ]:
import os
import sys

In [ ]:
def get_info_by_folder(folder_name, file_name=""):
    core_projects = {
        "SmartLife_Toolbox.py": ("綜合生活工具箱實作", "模組化整合、CLI 互動介面設計"),
        "RealEstate_Investment_Analyzer.py": ("地產投資分析器", "數位拆解、財務複利、三角估價"),
        "Loop_Practice_Tool.py": ("多功能迴圈工具", "規律生成、質數判定、排序基礎"),
        "Smart_Award_System.py": ("智慧年度評選系統", "封裝邏輯、參數傳遞、系統整合"),
        "Smart_Inventory_System.py": ("電商庫存管理系統", "串列操作、搜尋與排序演算法")
    }

    if file_name in core_projects:
        return core_projects[file_name]

    # 自動解析 Task 檔名邏輯
    auto_feature = file_name.replace(".py", "").replace("_", " ").title() if "_" in file_name else "基礎語法練習"

    info = {
        "function": ("函式封裝練習", "參數傳遞、Return 回傳值應用"),
        "loop": ("迴圈演算法練習", "for/while 迭代、控制流程"),
        "if_else": ("邏輯判斷練習", "條件分歧、布林邏輯運算"),
        "list": ("資料結構與排序", "串列操作、演算法效率優化"),
        "variable_constant": ("基礎語法練習", "變數賦值、基本數學運算"),
        "utils": ("工具程式", "專案輔助腳本與自動化工具"),
        "multidimensional_list": (f"多維串列應用 - {auto_feature}", "矩陣運算、巢狀迴圈與表格處理"),
        "file_input": ("檔案處理練習", "File I/O 與讀寫安全檢核","檔案處理實戰", "例外處理、檔案讀寫安全檢核")
    }
    return info.get(folder_name, ("基礎 Python 練習", "語法熟悉與邏輯實作"))

In [ ]:
def process_single_file(file_path):
    """處理單一檔案的標註邏輯"""
    if not file_path.endswith(".py") or not os.path.exists(file_path):
        return

    file_name = os.path.basename(file_path)
    # 取得檔案所在的資料夾名稱
    parent_dir = os.path.dirname(os.path.abspath(file_path))
    folder_name = os.path.basename(parent_dir)
    
    # 如果在 learning 子目錄裡，抓上一層的名稱
    if folder_name == "learning":
        folder_name = os.path.basename(os.path.dirname(parent_dir))
    
    feature, point = get_info_by_folder(folder_name, file_name)
    docstring = f'"""\n檔名：{file_name}\n功能：{feature}\n學習重點：{point}\n"""\n'

    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    if lines and lines[0].strip().startswith('"""'):
        end_index = -1
        for i, line in enumerate(lines):
            if i > 0 and '"""' in line.strip():
                end_index = i
                break
        new_content = docstring + "".join(lines[end_index+1:]) if end_index != -1 else docstring + "".join(lines)
    else:
        new_content = docstring + "".join(lines)

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(new_content)
    print(f"✅ 已完成精確標註: [{folder_name}] {file_name}")

In [ ]:
def update_all_folders():

    # 取得根目錄
    base_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    # 定義要掃描的兩大母目錄
    root_targets = ["Project", "Learning"]
    for folder in root_targets:
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path): continue
        
        # 掃描根資料夾與 learning 子資料夾
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".py") and file != "add_docstrings.py":
                    process_single_file(os.path.join(root, file))

In [ ]:
if __name__ == "__main__":
    if len(sys.argv) > 1:
        target = sys.argv[1]
        if os.path.isfile(target):
            process_single_file(target)
        elif os.path.isdir(target):
            for f in os.listdir(target):
                if f.endswith(".py"):
                    process_single_file(os.path.join(target, f))
        else:
            print(f"❌ 找不到目標: {target}")
    else:
        print("💡 未指定目標，執行全域更新...")
        update_all_folders()